### Problem 1: Dynamic Time Warping
这个算法可以用来计算两个序列之间的距离，比如可以用来做语音识别、股票相似K线的计算，任意不等长序列之间的距离计算。这个问题也叫做sequential alignment. 其实就是用来找出两个序列的alignment, 可以用如下图来描述：
![alt text](image1.png "Title")


在DTW的实现中，需要考虑几点：
1. 起始点与终止点： 一般起始点采用(0,0), 终止点采用 (len(s), len(t))
2. Local continuity:  在局部上做alignment的时候可以稍微灵活一些，比如跳过1个value等。 下图展示的是一种方式：
![alt text](image2.png "Title")
3. Global Continuity:  从全局的角度设定的限制条件
4. 另外就是每一个路径的权重

In [10]:
import numpy as np
import sys

# 定义距离
def euc_dist(v1, v2):
    return np.abs(v1-v2)

# DTW的核心过程，实现动态规划
def dtw(s, t):
    """
    s: source sequence
    t: target sequence
    """
    m, n = len(s), len(t)
    dtw = np.zeros((m, n))
    dtw.fill(sys.maxsize)
    
    # 初始化过程
    dtw[0,0] = euc_dist(s[0], t[0])
    for i in range (1,m):
        dtw[i,0] = dtw[i-1,0] + euc_dist(s[i], t[0])
    for i in range (1,n):
        dtw[0,i] = dtw[0,i-1] + euc_dist(s[0], t[i])
    
    # 核心动态规划流程，此动态规划的过程依赖于上面的图
    for i in range(1, m): # dp[][]
        for j in range(max(1, i- 10), min(n, i+10)):
            cost = euc_dist(s[i], t[j])
            ds = []
            ds.append(cost+dtw[i-1, j])
            ds.append(cost+dtw[i,j-1])
            ds.append(2*cost+dtw[i-1,j-1])
            ds.append(3*cost + dtw[i-1,j-2] if j>1 else sys.maxsize)
            ds.append(3*cost+dtw[i-2,j-1] if i>2 else sys.maxsize)
            # pointer 
            dtw[i,j] = min(ds)
    
    return dtw[m-1, n-1]


dtw([5,6,9], [5,6,7])

3.0

### Problem  2：  Edit Distance (编辑距离）
编辑距离用来计算两个字符串之间的最短距离，这里涉及到三个不通过的操作，add, delete和replace. 每一个操作我们假定需要1各单位的cost. 

例子： "apple", "appl" 之间的编辑距离为1 （需要1个删除的操作）
spell correction
"machine", "macaide" dist = 2
"mach", "aaach"  dist=2

s1 s2 s3 s4

t1 t2 t3 t4 t5

if s4==t5:
    return edit_dist([s1,s2,s3], [t1,t2,t3,t4])
    
if s4!=s5:
    1. replace s4 replaced by t5:
        return edit_dist([s1,s2,s3], [t1,t2,t3,t4]) + 1
    2. add
    3. delete

In [8]:
# 基于动态规划的解法
def edit_dist(str1, str2):
    
    # m，n分别字符串str1和str2的长度
    m, n = len(str1), len(str2)
    
    # 构建二位数组来存储子问题（sub-problem)的答案 
    dp = [[0 for x in range(n+1)] for x in range(m+1)] 
      
    # 利用动态规划算法，填充数组
    for i in range(m+1): 
        for j in range(n+1): 
  
            # 假设第一个字符串为空，则转换的代价为j (j次的插入)
            if i == 0: 
                dp[i][j] = j    
              
            # 同样的，假设第二个字符串为空，则转换的代价为i (i次的插入)
            elif j == 0:
                dp[i][j] = i
            
            # 如果最后一个字符相等，就不会产生代价
            elif str1[i-1] == str2[j-1]: 
                dp[i][j] = dp[i-1][j-1] 
  
            # 如果最后一个字符不一样，则考虑多种可能性，并且选择其中最小的值
            else: 
                dp[i][j] = 1 + min(dp[i][j-1],        # Insert 
                                   dp[i-1][j],        # Remove 
                                   dp[i-1][j-1])      # Replace 
  
    return dp[m][n] 

In [9]:
edit_dist("apple", "app")

2